В этом ноутбуке представлен пример предсказания, а также развертывание сервиса по генерации аккордов.

In [2]:
import numpy as np
import polars as pl
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from tqdm import tqdm

import re
from typing import List, Dict, Any, Tuple, Optional, Mapping, Set, Self, NamedTuple, TypedDict
from utils import *

### Загружаем модель

In [3]:
def load_tokenizer(filepath):
    """Загружает токенайзер из файла"""
    tokenizer_data = torch.load(filepath)
    
    # Создаем новый токенайзер
    tokenizer = ChordTokenizer()
    
    # Восстанавливаем состояние
    tokenizer._vocab = tokenizer_data['_vocab']
    tokenizer.notes = tokenizer_data['notes']
    tokenizer.moods = tokenizer_data['moods']
    tokenizer.extensions = tokenizer_data['extensions']
    tokenizer.symbols = tokenizer_data['symbols']
    tokenizer.complex_chords = tokenizer_data['complex_chords']
    
    print(f"Токенайзер загружен из {filepath}")
    return tokenizer
loaded_tokenizer = load_tokenizer('chord_tokenizer.pth')

Токенайзер загружен из chord_tokenizer.pth


In [4]:
VOCAB_SIZE = len(loaded_tokenizer.vocab)
BATCH_SIZE = 128
MAX_SEQ_LEN = 50
N_LAYERS = 6
EMBEDDING_SIZE = 64
NUM_HEADS = 8
HEAD_EMBEDDING_SIZE = EMBEDDING_SIZE // NUM_HEADS
FCCN_HIDDEN_SIZE = EMBEDDING_SIZE * 4
n_epoch = 10

In [5]:
def load_model(filepath, encoder_class, encoder_config):
    """Загружает модель из файла"""
    checkpoint = torch.load(filepath, map_location='cpu')
    
    encoder = encoder_class(**encoder_config)
    model = BERTLM(encoder)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    
    print(f"Модель загружена из {filepath}")
    
    return model


encoder_config = {
    'vocab_size': VOCAB_SIZE,
    'n_layers': N_LAYERS,
    'embedding_size': EMBEDDING_SIZE,
    'num_heads': NUM_HEADS,
    'head_embedding_size': HEAD_EMBEDDING_SIZE,
    'fcnn_hidden_size': FCCN_HIDDEN_SIZE,
}

loaded_model = load_model('/root/chord/chord_bert_model.pth', Encoder, encoder_config)

Модель загружена из /root/chord/chord_bert_model.pth


In [6]:
loaded_model.eval()

BERTLM(
  (_encoder): Encoder(
    (_embeddings): BERTEmbedding(
      (_embeddings): Embedding(1213, 64, padding_idx=0)
      (_segment_embeddings): Embedding(3, 64, padding_idx=0)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (_positional_embeddings): RotaryPositionEmbedding()
    (_layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (_mha): RoPEMultiHeadedAttention(
          (_positional_embedding): RotaryPositionEmbedding()
          (_Q): Linear(in_features=64, out_features=64, bias=True)
          (_K): Linear(in_features=64, out_features=64, bias=True)
          (_V): Linear(in_features=64, out_features=64, bias=True)
          (_W_proj): Linear(in_features=64, out_features=64, bias=True)
          (_dropout): Dropout(p=0.1, inplace=False)
          (_layernorm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (_fcnn): FCNNBlock(
          (_linear1): Linear(in_features=64, out_features=256, bias=False)
          (_linear2): Linear(in_f

In [7]:
def predict_masked_chord(model, tokenizer, chord_sequence):
    """
    Предсказывает аккорд на месте знака вопроса в последовательности
    Пример: ["A", "B", "?", "B", "D"] -> предсказывает аккорд вместо '?'
    """

    print(f'Input {chord_sequence}')
    chord_sequence = chord_sequence.split()
    masked_sequence = [chord if chord != '?' else tokenizer._mask_token for chord in chord_sequence]
    if tokenizer._mask_token not in masked_sequence:
        masked_sequence.append(tokenizer._mask_token)

    input_ids = tokenizer.tokenize_ids(masked_sequence)
    input_ids = [i[1] for i in input_ids]


    inputs = torch.tensor([input_ids])

    mask_index = input_ids.index(tokenizer.mask_token_id)
    segment_label = torch.zeros_like(inputs)
    # Предсказание
    with torch.no_grad():
        outputs = model(inputs, segment_label)
        predictions = outputs.logits if hasattr(outputs, 'logits') else outputs

    predicted_index = torch.argmax(predictions[1][0][mask_index]).item()
    predicted_chord = tokenizer.vocab.get(predicted_index, tokenizer._unknown_token)
    masked_sequence[mask_index] = predicted_chord
    separator = " "
    result = separator.join(masked_sequence)
    print(f'Result {result}')
    return predicted_chord, result, predictions

In [8]:
predicted_chord, result, predictions = predict_masked_chord(loaded_model, loaded_tokenizer, 'D C D F B B F B B F B ? F D F C D D C D D F')

Input D C D F B B F B B F B ? F D F C D D C D D F
Result D C D F B B F B B F B G F D F C D D C D D F


### Поднимаем gradio

In [9]:
import gradio as gr
import numpy as np
import io
import wave
import random


# Синтез гитарного звука 
SAMPLE_RATE = 22050

NOTE_FREQS = {
    'C': 261.6, 'C#': 277.2, 'Db': 277.2,
    'D': 293.7, 'D#': 311.1, 'Eb': 311.1,
    'E': 329.6, 'F': 349.2, 'F#': 370.0,
    'Gb': 370.0, 'G': 392.0, 'G#': 415.3,
    'Ab': 415.3, 'A': 440.0, 'A#': 466.2,
    'Bb': 466.2, 'B': 493.9
}

INTERVALS = {
    'maj': [0, 4, 7],
    'min': [0, 3, 7],
    'dim': [0, 3, 6],
    'aug': [0, 4, 8],
    '7': [0, 4, 7, 10],
    'maj7': [0, 4, 7, 11],
    'm7': [0, 3, 7, 10],
    'sus2': [0, 2, 7],
    'sus4': [0, 5, 7],
    '5': [0, 7]
}

def parse_chord(chord):
    chord = chord.strip()
    root = ''
    quality = ''
    for note in sorted(NOTE_FREQS.keys(), key=lambda x: -len(x)):
        if chord.startswith(note):
            root = note
            quality = chord[len(note):]
            break
    if quality == '':
        quality = 'maj'
    if quality in ['m', 'min']:
        quality = 'min'
    elif quality in ['maj', '']:
        quality = 'maj'
    if quality not in INTERVALS:
        quality = 'maj'
    return root, quality

def chord_to_wave_guitar(chord, duration=1.1):
    root, quality = parse_chord(chord)
    base_freq = NOTE_FREQS.get(root, 261.6)
    intervals = INTERVALS[quality]

    t = np.linspace(0, duration, int(SAMPLE_RATE * duration), False)
    wave_data = np.zeros_like(t)
    for interval in intervals:
        freq = base_freq * 2 ** (interval / 12)
        envelope = np.exp(-3 * t)  # гитарное затухание
        wave_data += 0.3 * np.sin(2 * np.pi * freq * t) * envelope

    wave_data = wave_data / np.max(np.abs(wave_data))
    pcm = (wave_data * 32767).astype(np.int16)

    buf = io.BytesIO()
    with wave.open(buf, "wb") as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(SAMPLE_RATE)
        wf.writeframes(pcm.tobytes())
    return buf.getvalue()

def sequence_to_audio(chords: str):
    chunks = [chord_to_wave_guitar(ch) for ch in chords.split()]
    full = io.BytesIO()
    with wave.open(full, "wb") as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(SAMPLE_RATE)
        for w in chunks:
            wf.writeframes(wave.open(io.BytesIO(w)).readframes(99999999))
    return full.getvalue()


def predict_chords(user_input):
    result_chords = predict_masked_chord(loaded_model, loaded_tokenizer, user_input)[1]
    audio_bytes = sequence_to_audio(result_chords)
    return result_chords, (SAMPLE_RATE, np.frombuffer(audio_bytes, dtype=np.int16))

iface = gr.Interface(
    fn=predict_chords,
    inputs=[
        gr.Textbox(label="Введите аккорды", placeholder="Am ? Dm E")
    ],
    outputs=[
        gr.Textbox(label="Сгенерированные аккорды"),
        gr.Audio(label="Аудио", type="numpy")
    ],
    title="🎸 Генератор аккордов", 
    allow_flagging='never'
)

iface.launch()

/root/venv/lib/python3.12/site-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Input D C D F B B F B B F B B F ? F C D D C D F
Result D C D F B B F B B F B B F C F C D D C D F
